In [1]:
import nltk
import re
import pandas as pd
from nltk.corpus import stopwords

In [2]:
reviews = pd.read_csv('yelp__review.csv', low_memory=False)

In [3]:
reviews_df = reviews[['business_id','text','date']]

In [4]:
reviews_df.tail()

,business_id,text,date
5996992,abIF7pw1Hyu-eVW3LwNhyg,I must admit i have had very bad experience he...,2016-02-29
5996993,fbLYRHIZAt3q839whhaBUg,Worst service experience in awhile. I usually ...,2017-07-06
5996994,b_8jCti0vuouJ_fjZpdmZw,"Loved the location, the patio, the service, an...",2018-04-14
5996995,wY3dlpLa0BHfDENYAPYuZA,I absolutely hate this place. I would give it ...,2018-04-07
5996996,6E4i0NHTRAgpwbbRwZhlnw,The first and the last time I will go there. M...,2018-06-07


In [5]:
reviews_df['text'][97]

'STAY AWAY!!! Because I moved this was the closest to my new home. I had my oil and filter changed here about a month ago and after a week my "oil change" light came on. My bf mentioned maybe the guy just forgot to reset it so he reset it for me.... Last week it came on again and turned off after a few hours... the next day it was on all day... my bf decided to change the oil and filter himself and showed me that someone wrote on the filter to make sure we know it was changed and the date said JULY 2016!!!!! (see picture posted) This place hasn\'t changed my filter the both times I\'ve come to them after telling them I need an oil and filter change. Both times I was also told it will take about 40min.... but only took 10.... did they even check anything else?  And I love how both times They asked if I was paying cash and when I told them cash, both times they\'re machine/computer was "down" so they were unable to print me a receipt...what\'re you guys doing in there???'

In [ ]:
reviews_df['text_length'] = reviews_df['text'].apply(len)

In [12]:
import string
nltk.download('stopwords')
def load_stopwords():
    return set(stopwords.words("english"))
main_count = 0
def pre_process(text):
    global main_count
    main_count+=1
    count = 0
    porter = nltk.stem.PorterStemmer()
    text = text.lower() if text.isalpha() else text
    remove = string.punctuation
    remove = remove.replace("?", "") # don't remove ?
    remove = remove.replace("!","") # don't remove !
    pattern = r"[{}]".format(remove)
    text = re.sub(pattern, " ", text)
    words = text.split()
    stops = load_stopwords()
    meaningful_words = [porter.stem(w) for w in words if not w in stops]
    if main_count%100000 == 0:
        print(main_count)
    return " ".join(meaningful_words)

reviews_df['text'] = reviews_df['text'].apply(lambda x: pre_process(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deept\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'234 gtudt'

In [8]:
reviews_df['text'][97]

'STAY AWAY!!! Because I moved this was the closest to my new home. I had my oil and filter changed here about a month ago and after a week my "oil change" light came on. My bf mentioned maybe the guy just forgot to reset it so he reset it for me.... Last week it came on again and turned off after a few hours... the next day it was on all day... my bf decided to change the oil and filter himself and showed me that someone wrote on the filter to make sure we know it was changed and the date said JULY 2016!!!!! (see picture posted) This place hasn\'t changed my filter the both times I\'ve come to them after telling them I need an oil and filter change. Both times I was also told it will take about 40min.... but only took 10.... did they even check anything else?  And I love how both times They asked if I was paying cash and when I told them cash, both times they\'re machine/computer was "down" so they were unable to print me a receipt...what\'re you guys doing in there???'

TypeError: object of type 'float' has no len()

In [ ]:
reviews_df.tail()

In [ ]:
reviews_df['count_punctuation'] = reviews_df.text.str.count('\?|\!')

In [ ]:
reviews_df.tail()

In [ ]:
from nltk import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('punkt')
def find_sentiment_score(text):
    tokens = word_tokenize(text)
    sia = SentimentIntensityAnalyzer()
    pos_count = 0
    neg_count = 0
    for w in tokens:
        if(sia.polarity_scores(w)['compound']) >= 0.5:
            pos_count+=1
        elif(sia.polarity_scores(w)['compound']) <= -0.5:
            neg_count += 1
    return pos_count - neg_count
# reviews_df['sentiment_score'] = reviews_df['text'].apply(lambda x: find_sentiment_score(x))
    

In [ ]:
reviews_df.tail()

In [ ]:
feature_names = pd.Series(' '.join(reviews_df['text']).lower().split()).value_counts()[:1000]

In [ ]:
feature_names = feature_names.keys().values.tolist()

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# v = TfidfVectorizer(vocabulary=feature_names)
# x = v.fit_transform(reviews_df['text'])
# # print(v.get_feature_names())
# reviews_df['tfidf'] = list(x.toarray())
# reviews_df.tail()

In [ ]:
text_df = reviews_df.groupby(reviews_df.business_id).agg(lambda x: ' '.join(x.unique())).reset_index()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(vocabulary=feature_names)
x = v.fit_transform(text_df['text'])
print(v.get_feature_names())
text_df['tfidf_unigram'] = list(x.toarray())
text_df.tail()

In [ ]:
text_df['sentiment_score'] = text_df['text'].apply(lambda x: find_sentiment_score(x))

In [ ]:
text_df.loc[text_df['business_id'] == '7wHLFohwCw8l6WS-feLjeg', 'tfidf'].item()

In [ ]:
reviews_df[0:2]
reviews_df.loc[reviews_df['business_id'] == '7wHLFohwCw8l6WS-feLjeg', 'text_length'].item()

In [ ]:
df_1 = reviews_df.groupby('business_id').agg({'text_length': 'mean', 'count_punctuation': 'mean'}).reset_index()

In [ ]:
df_1.loc[text_df['business_id'] == '7wHLFohwCw8l6WS-feLjeg', 'text_length'].item()

In [ ]:
df_1[0:2]

In [ ]:
text_df[0:2]

In [ ]:
final_df_unigram = pd.merge(df_1, text_df, on='business_id')

In [ ]:
final_df_unigram[0:2]

In [ ]:
final_df_unigram.to_csv('unigram.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2,2))
n_grams = vectorizer.fit_transform(reviews_df['text'])

In [ ]:
vocab = vectorizer.vocabulary_
count_values = n_grams.toarray().sum(axis=0)
counts = sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)[:500]

In [ ]:
vocabulary = list(x[1] for x in counts)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(ngram_range=(2,2), max_features=500)
x = v.fit_transform(text_df['text'])
# print(v.get_feature_names())
text_df['tfidf_bigram'] = list(x.toarray())
text_df.tail()

In [ ]:
final_df_bigram = pd.merge(df_1, text_df, on='business_id')

In [ ]:
final_df_bigram.to_csv('bigram.csv')